In [2]:
import pypandoc
import requests
from bs4 import BeautifulSoup
import unicodedata
import re

In [84]:
output=pypandoc.convert_file('testdocs/Chemical Kinetics 02.docx','html')
soup=BeautifulSoup(output,'html.parser')

[WARNING] Could not convert TeX math \begin{matrix}
  k & \  = \frac{0.693}{t_{1/2}} = \frac{0.693}{1386} \\
   & \  = 0.5 \times 10^{- 3}{\text{\ }s}^{- 1} \\
  \end{matrix}, rendering as TeX
[WARNING] Could not convert TeX math k = \frac{2.303}{t}\log_{10}\frac{a}{(a - x)}, rendering as TeX
[WARNING] Could not convert TeX math k = \frac{2.303}{t}\log_{10}\frac{a}{(a - x)}, rendering as TeX
[WARNING] Could not convert TeX math k_{1} = \frac{2.303}{t_{1}}log\frac{a_{1}}{a_{1} - x_{1}}, rendering as TeX
[WARNING] Could not convert TeX math k_{2} = \frac{2.303}{t_{2}}log\frac{a_{2}}{a_{2} - x_{2}}, rendering as TeX
[WARNING] Could not convert TeX math x_{1} = \frac{60}{100}a_{1},t_{1} = 60, rendering as TeX
[WARNING] Could not convert TeX math x_{2} = \frac{50}{100}a_{2},t_{2} =, rendering as TeX
[WARNING] Could not convert TeX math \frac{2.303}{t_{1}}log\frac{a_{1}}{a_{1} - x_{1}} = \frac{2.303}{t_{2}}log\frac{a_{2}}{a_{2} - x_{2}}, rendering as TeX
[WARNING] Could not convert TeX math 

In [85]:
print(soup.prettify())

<table>
 <colgroup>
  <col style="width: 50%"/>
  <col style="width: 49%"/>
 </colgroup>
 <thead>
  <tr class="header">
   <th>
    Division
   </th>
   <th>
    12th
   </th>
  </tr>
 </thead>
 <tbody>
  <tr class="odd">
   <td>
    Subject
   </td>
   <td>
    Chemistry
   </td>
  </tr>
  <tr class="even">
   <td>
    Chapter
   </td>
   <td>
    Chemical Kinetics
   </td>
  </tr>
  <tr class="odd">
   <td>
    Author
   </td>
   <td>
    Ruhani kashni
   </td>
  </tr>
  <tr class="even">
   <td>
    Category
   </td>
   <td>
    02
   </td>
  </tr>
 </tbody>
</table>
<table>
 <colgroup>
  <col style="width: 100%"/>
 </colgroup>
 <thead>
  <tr class="header">
   <th>
    <p>
     Which among the following represent intercepted in Arrhenius
plot
    </p>
    <p>
     (2012)
    </p>
   </th>
  </tr>
 </thead>
 <tbody>
  <tr class="odd">
   <td>
    <blockquote>
     <p>
      Ln A
     </p>
    </blockquote>
   </td>
  </tr>
  <tr class="even">
   <td>
    <blockquote>
     <p>
      

In [86]:
config=soup.find_all('table')[0]
questions=soup.find_all('table')[1:]
no_questions=len(questions)
no_questions

48

In [87]:
# config=pd.read_html(str(config))[0]
# config

In [88]:
question_set={}
format=["Question","Option A","Option B","Option C","Option D","Correct Answer","Hint","Explanation","Sub-topic"]

In [89]:
rows = config.find_all('tr')
table_dict = {cols[0].text: cols[1].text for row in rows for cols in [row.find_all(['th', 'td'])] if cols}
# print(table_dict)
question_set["Config"]=table_dict
question_set

{'Config': {'Division': '12th',
  'Subject': 'Chemistry',
  'Chapter': 'Chemical Kinetics',
  'Author': 'Ruhani kashni',
  'Category': '02'}}

In [90]:
print(questions[47])

<table>
<colgroup>
<col style="width: 100%"/>
</colgroup>
<thead>
<tr class="header">
<th>According to the Arrhenius equation, as the temperature increases,
the rate constant (k) of a reaction:</th>
</tr>
</thead>
<tbody>
<tr class="odd">
<td>Decreases.</td>
</tr>
<tr class="even">
<td>Remains constant.</td>
</tr>
<tr class="odd">
<td>Increases.</td>
</tr>
<tr class="even">
<td>Depends on the activation energy.</td>
</tr>
<tr class="odd">
<td>c</td>
</tr>
<tr class="even">
<td>An increase in temperature leads to an increase in the rate constant
(k) of a reaction.</td>
</tr>
<tr class="odd">
<td>According to the Arrhenius equation, an increase in temperature
leads to an increase in the rate constant (k) of a reaction. This is
because higher temperatures provide reactant molecules with greater
kinetic energy, leading to more frequent and energetic collisions, which
increase the rate of the reaction</td>
</tr>
<tr class="even">
<td>Arrhenius equation</td>
</tr>
</tbody>
</table>


In [125]:
def extract_data(question):
    question_dict = {}
    format = ["Question", "Option A", "Option B", "Option C", "Option D", "Correct Answer", "Hint", "Explanation", "Sub-topic"]

    # Extract the question
    # p_tags = question.find('th').find_all('p')
    # question_text = ' '.join([p.text for p in p_tags[:-1]])  # Join all p tags except the last one (year)
    question_text=question.find('tr',class_='header').text.strip()
    question_text = re.sub(r'\(\d+\)', '', question_text)  # Remove the year

    question_dict[format[0]] = question_text.strip().replace('\n', ' ')

    # Extract the options
    options = question.find_all('td')
    for i, option in enumerate(options[:5], 1):  # Only the first 5 td tags are options
        if option.find('blockquote'):  # If the option is in a blockquote tag
            text = option.find('blockquote').text.strip().replace('\n', ' ')
        elif option.find('p'):  # If the option is in a p tag
            text = option.find('p').text.strip().replace('\n', ' ')
        else:  # If the option is directly in the td tag
            text = option.text.strip().replace('\n', ' ')
        question_dict[format[i]] = text

    # Extract the correct answer
    correct_answer = options[5].text.strip().replace('\n', ' ')
    question_dict[format[5]] = correct_answer

    # Extract the hint (assuming it's the text in the 7th td tag)
    hint = options[6].text.strip().replace('\n', ' ')
    question_dict[format[6]] = hint

    # Extract the explanation
    explanation = options[6]  # Explanation is in multiple p tags
    explanation_text = ' '.join([p.text for p in explanation]).strip().replace('\n', ' ')
    question_dict[format[7]] = explanation_text

    # Extract the sub-topic
    sub_topic = options[-1].text.strip().replace('\n', ' ')
    question_dict[format[8]] = sub_topic

    return question_dict

extract_data(questions[4])


{'Question': 'The given reaction is 2N₂O₅ ⇔2N₂O₄ + O₂',
 'Option A': 'Unimolecular',
 'Option B': 'Multimolecular',
 'Option C': 'Bimolecular',
 'Option D': 'None of the above',
 'Correct Answer': 'The number of reacting species taking part in an elementary reaction which must collide simultaneously in order to bring about chemical reaction is called molecularity of a reaction.',
 'Hint': 'The given reaction 2N₂O₅ ⇔2N₂O₄ + O₂ It is bimolecular and first order reaction because rate is directly proportional to the reactant [N₂O₅].',
 'Explanation': 'The given reaction 2N₂O₅ ⇔2N₂O₄ + O₂   It is bimolecular and first order reaction because rate is directly proportional to the reactant [N₂O₅].',
 'Sub-topic': 'Instantaneous rate'}

In [104]:
question_set["Questions"]=[extract_data(question) for question in questions]

In [105]:
extract_data(questions[4])

# counter=0
# for question in questions:
#     counter+=1
#     print(counter)
#     print(extract_data(question))

''

In [69]:
sel_question=questions[47]
# def extqs(sel_question):
# Extract the question
# ques=sel_question.find('thead').find('p').text.strip().replace('\n', ' ') 
# for i in questions:
p_tags = sel_question.find('th').find_all('p')
ques = ' '.join([p.text for p in p_tags[:-1]])  # Join all p tags except the last one (year) 
ques = re.sub(r'\(\d+\)', '', ques)  # Remove the year 
# print(f"Question {questions.index(i)} : {ques}") 

opta=sel_question.find('tbody').find_all('tr')[0].text.strip().replace('\n', ' ')
optb=sel_question.find('tbody').find_all('tr')[1].text.strip().replace('\n', ' ')
optc=sel_question.find('tbody').find_all('tr')[2].text.strip().replace('\n', ' ')
optd=sel_question.find('tbody').find_all('tr')[3].text.strip().replace('\n', ' ')
coropt=sel_question.find('tbody').find_all('tr')[4].text.strip().replace('\n', ' ')
hint=sel_question.find('tbody').find_all('tr')[5].text.strip().replace('\n', ' ') 
exp=sel_question.find('tbody').find_all('tr')[6].text.strip().replace('\n', ' ') 
subt=sel_question.find('tbody').find_all('tr')[7].text.strip().replace('\n', ' ')
data=[ques,opta,optb,optc,optd,coropt,hint,exp,subt]

for i in questions:
    print(f"""Question {questions.index(i)} : {i.find('tr',class_='header').text.strip() }  """ )

Question 0 : Which among the following represent intercepted in Arrhenius
plot
(2012)  
Question 1 : Half-life period in first order reaction depends upon
(2011)  
Question 2 : What would be the unit of k in the given reaction
K
A +B → C and the given reaction is a homogeneous reaction
(2015)  
Question 3 : Which among the following can increase the rate constant of
reaction
(2010)  
Question 4 : The given reaction is
2N₂O₅ ⇔2N₂O₄ + O₂
(2008)  
Question 5 : Calculate the half-life period of a first order reaction where
specific rate constant is 2min⁻¹
(2003)  
Question 6 : In Arrhenius plot slope Is
(2002)  
Question 7 : Find the time taken for completing half reaction if the initial
concentration of zero order reaction is reduced to 1/4thh
(2004)  
Question 8 : For the zero-order reaction rate constant is equals to
(2017)  
Question 9 : Half-life for second order reaction is
(2003)  
Question 10 : Rate constant for first order reaction is 2.6 × 10⁻¹⁴ s⁻¹. Find
the half-life period of 

In [21]:
question_set=dict(zip(format,extqs(sel_question)))

In [26]:
# question_set
extqs(sel_question)

['The given reaction is', 'Unimolecular', 'Multimolecular', 'Bimolecular', 'None of the above', 'c', 'The number of reacting species taking part in an elementary reaction which must collide simultaneously in order to bring about chemical reaction is called molecularity of a reaction.', 'The given reaction 2N₂O₅ ⇔2N₂O₄ + O₂ It is bimolecular and first order reaction because rate is directly proportional to the reactant [N₂O₅].', 'Instantaneous rate']


In [23]:
questions[4].find('tbody')

<tbody>
<tr class="odd">
<td><blockquote>
<p>Unimolecular</p>
</blockquote></td>
</tr>
<tr class="even">
<td><blockquote>
<p>Multimolecular</p>
</blockquote></td>
</tr>
<tr class="odd">
<td><blockquote>
<p>Bimolecular</p>
</blockquote></td>
</tr>
<tr class="even">
<td><blockquote>
<p>None of the above</p>
</blockquote></td>
</tr>
<tr class="odd">
<td>c</td>
</tr>
<tr class="even">
<td>The number of reacting species taking part in an elementary reaction
which must collide simultaneously in order to bring about chemical
reaction is called molecularity of a reaction.</td>
</tr>
<tr class="odd">
<td><p>The given reaction 2N₂O₅ ⇔2N₂O₄ + O₂</p>
<p>It is bimolecular and first order reaction because rate is directly
proportional to the reactant [N₂O₅].</p></td>
</tr>
<tr class="even">
<td>Instantaneous rate</td>
</tr>
</tbody>